In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
dev_data = torch.load('../condqa_files/data/dev_inputs')


In [2]:
dev_data[0]

[tensor([50279, 50280, 50265,  ...,     1,     1,     1]),
 tensor([ True,  True,  True,  ..., False, False, False]),
 tensor([ True,  True,  True,  ..., False, False, False]),
 tensor([ True,  True,  True,  ..., False, False, False]),
 tensor([-1, -1, -1,  ...,  0,  0,  0], dtype=torch.int16),
 tensor([ 1, -1, -1,  ...,  0,  0,  0], dtype=torch.int16),
 tensor([[-1, -1],
         [ 0,  0],
         [ 0,  0],
         ...,
         [ 0,  0],
         [ 0,  0],
         [ 0,  0]], dtype=torch.int16),
 tensor([[[ True, False],
          [False, False],
          [False, False],
          [False, False],
          [False, False]],
 
         [[False, False],
          [False, False],
          [False, False],
          [False, False],
          [False, False]],
 
         [[False, False],
          [False, False],
          [False, False],
          [False, False],
          [False, False]],
 
         ...,
 
         [[False, False],
          [False, False],
          [False, False],
  

In [2]:

import torch
import os
import argparse
from utils import input_to_batch, Tokenizer, init_logger
from model.modelling_hpt import HPTModel

train_inputs = torch.load('../condqa_files/data/dev_inputs')
train_inputs = [[j.cuda() for j in i] for i in train_inputs]
train_inputs = input_to_batch(train_inputs, batch_size = 1, distributed = False)
class Config:
    def __init__(self):
        pass
config = Config()
config.contrastive_learning = False
config.accumulation_step = 4
config.logdir='test'
config.warmup_epoch_num=10
config.total_epoch_num=100
config.model_hidden_size=768
config.ha_num_heads=12
config.ha_hidden_size=64
config.output_file='outputs/output'
config.inference_data='dev_data'
config.tqdm=True
config.model_root='../condqa_files/model/'
config.data_root='../condqa_files/data/'
config.epoch=0
config.mode='train'

model = HPTModel(config)
model = model.cuda()


Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
import torch
from utils import Tokenizer
train_inputs = torch.load('../condqa_files/data/train_inputs')
tokenizer = Tokenizer('../condqa_files/model')


In [20]:
for i in train_inputs:
    if i[-2][:, :, 1].count_nonzero(0).count_nonzero() == 2 :
        print(tokenizer.decode(i[0][i[5]==1]))
        print(i[-1])

<p><p><p><p>
tensor(6)
<p><p><p>
tensor(36)
<li><li>
tensor(58)
<li><li><p>
tensor(62)
<yes><no>
tensor(84)
<p><p>
tensor(108)
<yes><no>
tensor(116)
<yes><no>
tensor(186)
<tr><tr><tr>
tensor(250)
<p><p>
tensor(253)
<yes><no>
tensor(380)
<yes><no>
tensor(594)
<yes>
tensor(599)
<li><li>
tensor(722)
<yes><no>
tensor(729)
<p><p>
tensor(743)
<p>
tensor(761)
<li><li>
tensor(767)
<li><li><p><p>
tensor(819)
<yes><no>
tensor(825)
<p><tr>
tensor(862)
<yes><no>
tensor(863)
<yes><no>
tensor(867)
<p><p>
tensor(888)
<yes><no>
tensor(916)
<yes><no>
tensor(938)
<yes><no>
tensor(949)
<p><p><p>
tensor(971)
<p><p>
tensor(979)
<li><li>
tensor(996)
<p><p><p><p>
tensor(1038)
<tr><tr>
tensor(1072)
<tr><tr>
tensor(1087)
<li><li><li><li>
tensor(1096)
<yes><no>
tensor(1104)
<yes><no>
tensor(1161)
<p><p>
tensor(1178)
<p><p><p>
tensor(1193)
<yes><no>
tensor(1232)
<yes><no>
tensor(1237)
<p>
tensor(1240)
<yes><no>
tensor(1255)
<yes><no>
tensor(1292)
<p><p><li><li>
tensor(1341)
<yes><no>
tensor(1352)
<yes><no>
tenso

In [6]:
(train_inputs[0][5] == 1).nonzero()

tensor([[0]])

In [11]:
for i in train_inputs:
    if (i[5] == 1).count_nonzero() == 2:
        print(tokenizer.decode(i[0][i[1]]), i[-1])

<yes><no> Document: <h1><h1><h2><p><li><h2><p><li>Title: Apply for a Gender Recognition Certificate Question: What age can I apply for a certificate? Scenario: I am a 16 year old living in Derby and was born male, I do not feel like I fit into any gender category. tensor(8)
<yes><no> Document: <h1><h1><p><li><li><h1><p><li><li><p>Title: Disabled Facilities Grants Question: Is she allowed to seek disabled facility support? Scenario: My aunt is disabled and owns a big home which needs light and heating system repairs. She is afraid of the house getting moulds and needs urgent repair. tensor(12)
<yes><no> Document: <h1><h1><p><h1><p>Title: Claim a deputyship fee refund Question: Do I need to make a claim to receive a refund, or is this automatic? Scenario: I am 64, live in Wales and since May 2013 have been the Personal Welfare Deputy for my mother, who has advanced dementia. I believe I was overcharged both for the initial assessment and for annual supervisions in subsequent years. tenso

In [29]:
import torch
from utils import Tokenizer
tokenizer = Tokenizer('../condqa_files/model')
train_inputs = torch.load('../condqa_files/data/dev_inputs')
# train_inputs = [[j.cuda() for j in i] for i in train_inputs]
# train_inputs = input_to_batch(train_inputs, batch_size = 1, distributed = False)

# for k in train_inputs:
#     model.encoder(k)

In [47]:
看看condition为什么还是冲不上去
想想yesno怎么进一步提高

<yes><no> Document: <h1>Property</h1><h2>Selling the property</h2><p>You don’t pay Capital Gains Tax when you sell your home. You do pay it if you make a profit when you sell a property that isn’t your main home.</p><p>If inheriting a property means you own 2 homes, you’ll have to nominate one of them as your main home. You must tell HMRC which property is your main home within 2 years of inheriting the property.</p>Title: Tax on property, money and shares you inherit Question: Will I pay capital gains tax if I sell my current home in order to move into an inherited property? Scenario: I recently inherited my childhood home in Sussex following the death of my father. For sentimental reasons, and because it is bigger than my current home, I would like to move into it with my family. This would mean either selling or renting out my current home. I know that you don't have to pay capital gains tax when you sell your own home, but I'm not clear whether my current home would be classed as m

In [25]:
import torch
def weighted_loss(input, target):
    """
    return the weighted loss considering the pos-neg distribution in target data
    """
    input = input.flatten()
    target = target.flatten()
    if target.shape[0] == 0:
        return torch.tensor(torch.nan).to(input.device)
    elif (target.count_nonzero() == 0) or (target.count_nonzero() == target.shape[0]):
        return torch.tensor(0).to(input.device)
    else:
        x = target.shape[0]/target.count_nonzero()/2
        y = target.shape[0]/(target.shape[0]-target.count_nonzero())/2
        tensor = torch.where(target == 1, x, y)
        loss_fn = torch.nn.BCELoss(weight=tensor)
        return loss_fn(input, target)


In [26]:
for i in torch.arange(0.01, 0.99, 0.01):
    print(i, weighted_loss(torch.tensor([i,i,i]), torch.tensor([0, 0, 0.])) + weighted_loss(torch.tensor([i,i,i]), torch.tensor([0, 0, 1.])))

tensor(0.0100) tensor(2.3076)
tensor(0.0200) tensor(1.9661)
tensor(0.0300) tensor(1.7685)
tensor(0.0400) tensor(1.6298)
tensor(0.0500) tensor(1.5235)
tensor(0.0600) tensor(1.4376)
tensor(0.0700) tensor(1.3659)
tensor(0.0800) tensor(1.3046)
tensor(0.0900) tensor(1.2511)
tensor(0.1000) tensor(1.2040)
tensor(0.1100) tensor(1.1619)
tensor(0.1200) tensor(1.1240)
tensor(0.1300) tensor(1.0897)
tensor(0.1400) tensor(1.0585)
tensor(0.1500) tensor(1.0298)
tensor(0.1600) tensor(1.0035)
tensor(0.1700) tensor(0.9791)
tensor(0.1800) tensor(0.9566)
tensor(0.1900) tensor(0.9357)
tensor(0.2000) tensor(0.9163)
tensor(0.2100) tensor(0.8982)
tensor(0.2200) tensor(0.8813)
tensor(0.2300) tensor(0.8655)
tensor(0.2400) tensor(0.8508)
tensor(0.2500) tensor(0.8370)
tensor(0.2600) tensor(0.8241)
tensor(0.2700) tensor(0.8120)
tensor(0.2800) tensor(0.8007)
tensor(0.2900) tensor(0.7902)
tensor(0.3000) tensor(0.7803)
tensor(0.3100) tensor(0.7711)
tensor(0.3200) tensor(0.7625)
tensor(0.3300) tensor(0.7546)
tensor(0.3

In [2]:
from model.modelling_hpt import HierarchicalTransformerAttention
class Config:
    def __init__(self):
        pass
config = Config()
config.model_hidden_size = 8
config.ha_num_heads = 2
config.ha_hidden_size = 4
model = HierarchicalTransformerAttention(config)

In [13]:
hidden_states[0,357]

tensor([0.5522, 0.3920, 0.0612, 0.0100, 0.7119, 0.2532, 0.5933, 0.2496])

In [15]:
import torch
hidden_states = torch.rand(1, 1000, 8)
level_hierarchy = torch.zeros(1, 1000, dtype=torch.long)
level_hierarchy[0,357] = 1
level_hierarchy[0,340] = 1
level_hierarchy[0,422] = 2
level_hierarchy[0,365] = 2
level_hierarchy[0,112] = 3
level_hierarchy[0,489] = 3
model(hidden_states, level_hierarchy)

tensor([[[0.6140, 0.5035, 0.1051,  ..., 0.4855, 0.2005, 0.3058],
         [0.8612, 0.0510, 0.4049,  ..., 0.3168, 0.2754, 0.6938],
         [0.6386, 0.5630, 0.0390,  ..., 0.2994, 0.6083, 0.9945],
         ...,
         [0.4540, 0.1962, 0.2060,  ..., 0.3976, 0.5115, 0.8663],
         [0.0926, 0.2572, 0.7409,  ..., 0.5588, 0.4912, 0.8088],
         [0.7868, 0.7610, 0.6261,  ..., 0.4590, 0.2714, 0.4650]]],
       grad_fn=<ScatterBackward0>)

In [14]:
import torch
from torch import nn
num_attn_heads = 2
embedding = nn.Embedding(14, num_attn_heads)
input = torch.tensor([[1, 3, 2, 1, 0], [3, 3, 1, 0, 0]])
nan = torch.nan
relationship_unmasked = input.unsqueeze(1) - input.unsqueeze(2)
mask = torch.tensor([[1, 1, 1, 1, 0], [1, 1, 1, 0, 0]])
mask = torch.einsum('ab,ac->abc', mask, mask)
relationship = relationship_unmasked - (1 - mask) * 1000
relationship += 6
relationship = torch.where(relationship > 0, relationship, 0)
batch_size, max_num_heads = relationship.shape[0], relationship.shape[1]
relationship = embedding(relationship.flatten()).reshape(batch_size, max_num_heads, max_num_heads, num_attn_heads)
relationship.shape


torch.Size([2, 5, 5, 2])

In [105]:
import torch
from model.modelling_hpt import HierarchicalTransformer
from utils import Tokenizer

model = HierarchicalTransformer()
tokenizer = Tokenizer('../condqa_files/model')


Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [119]:
input_ids = tokenizer(['<h1>want to say goodbye']).unsqueeze(0).cuda()
attn_masks = torch.tensor([[1, 1, 1, 1, 0]]).cuda()
global_masks = torch.tensor([[1, 0, 0, 1, 0]]).cuda()
model.cuda()
None

In [122]:
model(input_ids, attn_masks, global_masks)

torch.Size([1, 512, 768]) tensor([[[-0.0103,  0.0316, -0.0098,  ..., -0.0486,  0.0084, -0.1527],
         [ 0.1514,  0.4159,  0.0287,  ..., -0.6583,  0.0778,  0.0291],
         [-0.3224, -0.4276, -0.1158,  ...,  0.0882, -0.5146,  0.5798],
         ...,
         [-0.6967, -0.1657, -0.2636,  ..., -0.2160, -0.0507, -0.0555],
         [-0.6967, -0.1657, -0.2636,  ..., -0.2160, -0.0507, -0.0555],
         [-0.6967, -0.1657, -0.2636,  ..., -0.2160, -0.0507, -0.0555]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)
torch.Size([1, 512, 768]) tensor([[[ 4.8578e-02,  2.8375e-02, -1.9885e-03,  ..., -1.7267e-04,
          -2.2234e-03, -5.7610e-02],
         [ 3.2383e-01,  4.8041e-01, -1.7935e-01,  ..., -5.3644e-01,
           4.4969e-02,  4.5482e-01],
         [-6.9267e-02, -3.5137e-01, -3.0398e-01,  ...,  1.1198e-01,
          -3.4803e-01,  5.2847e-01],
         ...,
         [ 6.3892e-02, -7.6392e-02, -1.2592e-01,  ..., -2.5100e-01,
           5.1669e-02,  3.1363e-01],
         [ 6

tensor([[-0.0747,  0.0752, -0.0151,  ..., -0.1622, -0.0339, -0.0404],
        [-0.1478,  0.0336, -0.0080,  ..., -0.1127,  0.0643, -0.1872],
        [ 0.1026, -0.0994,  0.0026,  ..., -0.2137, -0.0058, -0.1224],
        [-0.0172,  0.0057,  0.0860,  ..., -0.2074,  0.0802, -0.0208],
        [ 0.0434,  0.1687, -0.3759,  ..., -0.2970, -0.1433,  0.0217]],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [123]:
import torch
inputs = torch.load('../condqa_files/data/train_inputs', map_location='cpu')


In [127]:
inputs[0]['labels'][3].shape

torch.Size([4000, 5, 2])

In [165]:
from torch import nn
attention = nn.MultiheadAttention(24, 3, 0.1, batch_first=True)
hidden_states = torch.ones(1, 5, 24)


In [166]:
hidden_states[0, -1, :] = 1
A = attention(hidden_states, hidden_states, hidden_states, key_padding_mask = torch.tensor([[False, False, False, False, True]]), )
print(A)
hidden_states[0, -1, :] = 2
A = attention(hidden_states, hidden_states, hidden_states, key_padding_mask = torch.tensor([[False, False, False, False, True]]), )
print(A)


(tensor([[[ 0.2209,  0.4147, -0.4436, -0.0619, -0.1164,  0.4118,  0.3615,
           0.3154, -0.5760, -0.3446, -0.7165,  0.6312,  0.0609,  0.0439,
           0.4782, -0.2106, -0.1674,  0.1571, -0.2756, -0.4135, -0.8233,
          -0.4418,  0.3761, -0.3517],
         [ 0.2209,  0.4147, -0.4436, -0.0619, -0.1164,  0.4118,  0.3615,
           0.3154, -0.5760, -0.3446, -0.7165,  0.6312,  0.0609,  0.0439,
           0.4782, -0.2106, -0.1674,  0.1571, -0.2756, -0.4135, -0.8233,
          -0.4418,  0.3761, -0.3517],
         [ 0.2209,  0.4147, -0.4436, -0.0619, -0.1164,  0.4118,  0.3615,
           0.3154, -0.5760, -0.3446, -0.7165,  0.6312,  0.0609,  0.0439,
           0.4782, -0.2106, -0.1674,  0.1571, -0.2756, -0.4135, -0.8233,
          -0.4418,  0.3761, -0.3517],
         [ 0.2209,  0.4147, -0.4436, -0.0619, -0.1164,  0.4118,  0.3615,
           0.3154, -0.5760, -0.3446, -0.7165,  0.6312,  0.0609,  0.0439,
           0.4782, -0.2106, -0.1674,  0.1571, -0.2756, -0.4135, -0.8233,
         

In [65]:
from utils import TxtNode, get_level
import pickle
from functools import reduce
import numpy
import random
from copy import deepcopy
import os


def split_tokens(tokens):
    tokens = tokens.tolist()
    cutoffs = []
    current_cutoff = []
    for idx, i in enumerate(tokens):
        if i not in range(50265, 50272):
            current_cutoff.append((i, idx))
        else:
            cutoffs.append(current_cutoff)
            current_cutoff = [(i, idx)]
    cutoffs.append(current_cutoff)
    return cutoffs

def create_node_from_document(document_tokens):
    base_node = TxtNode([(0, -1)], tokenizer)
    nodes = [base_node]
    current_node = base_node
    for segment in document_tokens:
        while current_node.level >= get_level(segment):
            current_node = current_node.parent

        child = TxtNode(segment, tokenizer)
        nodes.append(child)
        current_node.children.append(child)
        child.parent = current_node
        current_node = child
    return base_node

def get_grouped_tokens(input_ids):
    splited_tokens = split_tokens(input_ids)
    start_tokens = splited_tokens[0]
    document_tokens = splited_tokens[1:-1]
    end_tokens = [i for i in splited_tokens[-1] if i[0] != 0]
    l_document_tokens = []
    while True:
        token = end_tokens.pop(0)
        if token[0] in list(range(50272, 50279)):
            l_document_tokens.append(token)
            break
        else:
            l_document_tokens.append(token)
    document_tokens.append(l_document_tokens)
    return start_tokens, document_tokens, end_tokens


def repeat_a_node(base_node):
    nodes = base_node.get_nodes_list()
    nodes = [node for node in nodes if node.parent != None]
    node = random.choice(nodes)
    node_new = node.copy()
    parent = node.parent
    node_new.parent = parent
    position = random.choice(list(range(len(parent.children) + 1)))
    parent.children = parent.children[:position] + [node_new] + parent.children[position:]
    return base_node

def remove_a_node(base_node):
    nodes = base_node.get_nodes_list()
    nodes = [node for node in nodes if node.parent != None]
    node = random.choice(nodes)
    parent = node.parent
    parent.children.remove(node)
    return base_node

def mask_a_node(base_node):
    nodes = base_node.get_nodes_list()
    node = random.choice(nodes)
    node.text = [(50264, j) for i, j in node.text]
    return base_node

def reorder_a_node(base_node):
    nodes = base_node.get_nodes_list()
    nodes = [node for node in nodes if len(node.children) > 1]
    node = random.choice(nodes)
    random.shuffle(node.children)
    return base_node



def recover_index_from_node(node):
    nodes = node.get_nodes_list()
    text = [i.text for i in nodes][1:]
    origin_HTMLelement_index = [i[0][1] for i in text]
    text_lengths = [len(i) for i in text]
    generated_HTMLelement_index = [0] + numpy.cumsum(text_lengths)[:-1].tolist()
    HTMLelement_index = list(zip(origin_HTMLelement_index, generated_HTMLelement_index))
    text = reduce(lambda x, y: x + y, text)
    return text, HTMLelement_index



In [101]:
from time import time

def contrastive_sampling(input_ids):
    start_tokens, document_tokens, end_tokens = get_grouped_tokens(input_ids)
    base_node = create_node_from_document(document_tokens)
    for i in range(len(base_node.get_nodes_list()) // 20):
        base_node = repeat_a_node(base_node)

    for i in range(len(base_node.get_nodes_list()) // 20):
        base_node = remove_a_node(base_node)

    for i in range(len(base_node.get_nodes_list()) // 5):
        base_node = reorder_a_node(base_node)

    for i in range(len(base_node.get_nodes_list()) // 10):
        base_node = mask_a_node(base_node)

    document_tokens, HTMLelement_index = recover_index_from_node(base_node)
    HTMLelement_index = torch.tensor(HTMLelement_index)
    document_tokens = start_tokens + document_tokens + end_tokens
    HTMLelement_index[:, 1] += len(start_tokens)

    return document_tokens, HTMLelement_index

def generate_contrastive_sample(input):
    sample = inputs[0]
    input_ids = sample[0]
    global_mask = sample[1]
    attention_mask = sample[2]
    mask_HTMLelements = sample[3]
    mask_label_HTMLelements = sample[4]
    mask_answer_span = sample[5]
    qa_id = sample[6]
    mask_label_condition = sample[7]

    new_input, contrastive_pairs = contrastive_sampling(input_ids)
    new_input_ids = torch.tensor([i[0] for i in new_input])
    arrangement_index = torch.tensor([i[1] for i in new_input])

    global_mask = global_mask[arrangement_index]
    attention_mask = attention_mask[arrangement_index]
    mask_HTMLelements = mask_HTMLelements[arrangement_index]
    mask_label_HTMLelements = mask_label_HTMLelements[arrangement_index]
    mask_answer_span = mask_answer_span[arrangement_index]
    mask_label_condition = mask_label_condition[arrangement_index]

    text_length = new_input_ids.shape[0]
    new_input_ids = torch.concat((new_input_ids, torch.ones(4000 - text_length, dtype = torch.long)))
    global_mask = torch.concat((global_mask, torch.zeros(4000 - text_length, dtype = torch.bool)))
    attention_mask = torch.concat((attention_mask, torch.zeros(4000 - text_length, dtype = torch.bool)))
    mask_HTMLelements = torch.concat((mask_HTMLelements, torch.zeros(4000 - text_length, dtype = torch.bool)))
    mask_label_HTMLelements = torch.concat((mask_label_HTMLelements, torch.zeros((4000 - text_length, 3), dtype = torch.long)))
    mask_answer_span = torch.concat((mask_answer_span, torch.zeros((4000 - text_length, 2), dtype = torch.long)))
    mask_label_condition = torch.concat((mask_label_condition, torch.zeros((4000 - text_length, 5, 2), dtype = torch.bool)))

    new_sample = [new_input_ids, global_mask, attention_mask, mask_HTMLelements, mask_label_HTMLelements, \
        mask_answer_span, qa_id, mask_label_condition]
    return new_sample, contrastive_pairs


In [102]:
A = inputs[0]
B, pair = generate_contrastive_sample(A)

In [74]:
max_HTML_num = 3
ans_indicator = torch.tensor(
    [[[0, 1], [0, 0], [0, 0]]]
)

cond_indicator = torch.tensor(
    [[[1, 0], [1, 1], [0, 0]]]
)
ans_indicator = ans_indicator.transpose(-2, -1)
cond_indicator = cond_indicator.transpose(-2, -1)
cond_indicator = cond_indicator.unsqueeze(2).repeat(1, 1, max_HTML_num, 1)# 1, 2, 3, 3
label_condition = torch.einsum('abcd,abc->acd',cond_indicator, ans_indicator)
